In [1]:
%load_ext watermark
%watermark

2020-09-15T11:38:18-05:00

CPython 3.7.6
IPython 7.13.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.4.0-47-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
import pandas as pd
import numpy as np
datos = pd.read_csv("movies.csv")

In [3]:
datos.head()

,presupuesto,genero,lenguaje,popularidad,productores,pais,ventas,duracion,titulo,puntuacion,n_votos
0,NaN,Comedy,en,8.387519,Sandollar Productions,United States of America,76578911.0,106.0,Father of the Bride Part II,5.7,173.0
1,NaN,Drama,en,0.894647,Miramax,South Africa,676525.0,106.0,"Cry, the Beloved Country",6.7,13.0
2,3500000.0,Comedy,en,14.569650,New Line Cinema,United States of America,28215918.0,91.0,Friday,7.0,513.0
3,NaN,Comedy,en,8.963037,Paramount Pictures,United States of America,32.0,87.0,Black Sheep,6.0,124.0
4,12000000.0,Comedy,en,9.592265,Universal Pictures,United States of America,41205099.0,92.0,Happy Gilmore,6.5,767.0


In [4]:
from sklearn.impute import SimpleImputer
from sklearn import preprocessing, feature_extraction

In [5]:
columnas_num = datos.select_dtypes(["int","float"]).columns
columnas_num

Index(['presupuesto', 'popularidad', 'ventas', 'duracion', 'puntuacion',
       'n_votos'],
      dtype='object')

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline

In [7]:
imputador = SimpleImputer(strategy="mean")
escalador = preprocessing.StandardScaler()
    
class ColumnExtractor(TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
        
    def transform(self, X, **transform_params):
        return X[self.columns].values
    
    def fit(self, X, y=None, **fit_params):
        return self
    
transformador_numerico = Pipeline([
    ("selector", ColumnExtractor(columns=columnas_num)),
    ("imputador", imputador),
    ("escalador", escalador)
])

print("Tamaño: {}".format(transformador_numerico.fit_transform(datos).shape))
transformador_numerico.fit_transform(datos)

Tamaño: (1344, 6)


array([[-2.30934132e-16,  4.83540189e-01,  6.72892890e-01,
         1.83377052e-01, -2.35105859e-01, -1.75398320e-01],
       [-2.30934132e-16, -9.35413943e-01, -5.84099030e-01,
         1.83377052e-01,  4.52409968e-01, -5.00797371e-01],
       [-8.79400265e-01,  1.65427423e+00, -1.28029118e-01,
        -4.26257342e-01,  6.58664717e-01,  5.16074663e-01],
       ...,
       [-2.30934132e-16, -1.08389423e+00,  0.00000000e+00,
        -6.29468807e-01, -2.35105859e-01, -5.21134812e-01],
       [-2.30934132e-16,  7.40059020e-01,  0.00000000e+00,
        -4.66899635e-01, -2.35105859e-01, -4.01143912e-01],
       [-2.30934132e-16, -1.08174332e+00,  0.00000000e+00,
        -4.12470600e+00, -2.88511106e-02, -5.25202300e-01]])

In [8]:
estimator = LinearRegression()

col_numericas_independientes = datos.drop("ventas", axis=1).select_dtypes(np.number).columns

pipeline_completo = make_pipeline(
    ColumnExtractor(columns=col_numericas_independientes),
    SimpleImputer(strategy="mean"),
    escalador
)

In [9]:
pelis_con_ventas = datos[datos["ventas"].notnull()]

In [10]:
pipeline_estimador = make_pipeline(
    pipeline_completo,
    estimator
)

In [11]:
pipeline_estimador.fit(pelis_con_ventas[col_numericas_independientes], pelis_con_ventas["ventas"])

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnextractor',
                                  <__main__.ColumnExtractor object at 0x7f4103290a50>),
                                 ('simpleimputer', SimpleImputer()),
                                 ('standardscaler', StandardScaler())])),
                ('linearregression', LinearRegression())])

In [12]:
pipeline_estimador.predict(pelis_con_ventas[col_numericas_independientes])[:5]

array([27659884.87757905, 16691212.22384852, 34599464.48359309,
       22225253.19089289, 62509849.05316681])

In [13]:
from sklearn.model_selection import cross_val_score

resultados = cross_val_score(pipeline_estimador,
                            X=pelis_con_ventas,
                            y=pelis_con_ventas["ventas"],
                            scoring="neg_mean_absolute_error",
                            cv=5)

resultados

array([-31479522.91195995, -22551325.95234929, -20864221.66966091,
       -25286141.57398211, -18844821.79016568])

In [14]:
resultados.mean()

-23805206.77962359